# **Solution: Web Scraping and Data Extraction from a News Website using BeautifulSoup**
In this solution, we focus on web scraping using BeautifulSoup (BS4) and requests to extract relevant data from a news website. The task involves scraping multiple pieces of information from a series of news articles, which will include the article titles, publication dates, author names, content, and tags or keywords.

This solution demonstrates how to:
- Fetch the page content from a website using the requests library.
- Parse and extract the required data from the HTML page using BeautifulSoup (BS4).
- Handle pagination to scrape multiple pages of news articles.
- Clean and structure the scraped data for further use.
- Save the scraped data into a JSON file for easy analysis and sharing.


### Define the Scraping Script for BBC News
Now, let's write the code to scrape data from the BBC News website.

In [1]:
import requests
from bs4 import BeautifulSoup

# Define the URL of the BBC News website (example: World News section)
url = 'https://www.bbc.com/news/world'  # Change the URL for different sections

# Send HTTP request to the URL and fetch the response
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Page successfully fetched!")
else:
    print("Failed to retrieve page.")

Page successfully fetched!


### Parse HTML and Extract Data
Next, parse the HTML content with BeautifulSoup and extract data like article titles, publication dates, and links.

In [4]:
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Scrape article titles and publication links
article_titles = soup.find_all('h2', {'class': 'sc-8ea7699c-3 kwWByH'})  # Modify the class as per structure
article_links = [a['href'] for a in soup.find_all('a', href=True) if '/news/' in a['href']]

# Print titles and links
for title, link in zip(article_titles, article_links):
    print(f"Title: {title.text.strip()}")
    print(f"Link: https://www.bbc.com{link}")
    print("-" * 80)


Title: Israel and Hamas reach Gaza ceasefire and hostage release deal, sources tell BBC
Link: https://www.bbc.com/news/topics/c2vdnvdg6xxt
--------------------------------------------------------------------------------


### Scrape Additional Article Details (Content)
If you want to scrape more detailed content from each article, you need to follow the link to the individual article and extract the article content.

In [5]:
# Function to scrape article content from a given article URL
def scrape_article_content(article_url):
    article_response = requests.get(f"https://www.bbc.com{article_url}")
    if article_response.status_code == 200:
        article_soup = BeautifulSoup(article_response.text, 'html.parser')
        # Extract the article's main content (based on class or tag)
        content = article_soup.find('div', {'class': 'sc-8ea7699c-0 hcpAqY'})
        if content:
            return content.get_text().strip()
    return "No content found."

# Scrape content from the first article as an example
first_article_link = article_links[0]  # Pick the first article link
article_content = scrape_article_content(first_article_link)
print(f"Article Content: {article_content}")


Article Content: LIVEIsrael and Hamas reach Gaza ceasefire and hostage release deal, sources tell BBC


### Structuring the Data and Saving It
After scraping the data, you may want to structure the scraped information into a list of dictionaries and save it to a JSON file.

In [6]:
import json

# Create a list of articles with title, link, and content
articles_data = []
for title, link in zip(article_titles, article_links):
    article_data = {
        'title': title.text.strip(),
        'link': f"https://www.bbc.com{link}",
        'content': scrape_article_content(link),
    }
    articles_data.append(article_data)

# Save the data to a JSON file
with open('bbc_articles.json', 'w') as f:
    json.dump(articles_data, f, indent=4)

print("Data saved to bbc_articles.json")


Data saved to bbc_articles.json


### Displaying the Scraped Data
Finally, display the first few articles to ensure that the data is being scraped correctly.

In [7]:
# Display the first 3 articles from the scraped data
for article in articles_data[:3]:
    print(f"Title: {article['title']}")
    print(f"Link: {article['link']}")
    print(f"Content: {article['content'][:200]}...")  # Display first 200 characters of content
    print("-" * 80)

Title: Israel and Hamas reach Gaza ceasefire and hostage release deal, sources tell BBC
Link: https://www.bbc.com/news/topics/c2vdnvdg6xxt
Content: LIVEIsrael and Hamas reach Gaza ceasefire and hostage release deal, sources tell BBC...
--------------------------------------------------------------------------------


### Explanation of the Code:
- Requests: The `requests.get()` function fetches the HTML page content from the BBC News website.
- BeautifulSoup Parsing: The HTML content is parsed using BeautifulSoup to extract specific elements, such as article titles, links, and content.
Data Extraction:
- Article Titles: Extracted using `soup.find_all('h2', {'class': 'sc-8ea7699c-3 kwWByH'})`, which targets the titles of articles on the main page.
- Article Links: Extracted using `soup.find_all('a', href=True)`, filtering for links that contain `/news/` in the URL.
- Article Content: A separate function (`scrape_article_content()`) is used to scrape the full content of an article by following its link.
- Pagination Handling: This script does not include pagination, but you can extend it by iterating over multiple pages using the same method and constructing URLs for additional pages.
- Saving Data: The scraped data is structured into a list of dictionaries (`articles_data`), and saved in a JSON file for easy access and further processing.